# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770407514555                   -0.53    9.0         
  2   -2.771688072265       -2.89       -1.32    1.0    132ms
  3   -2.771714477624       -4.58       -2.52    1.0   97.0ms
  4   -2.771714674904       -6.70       -3.40    1.0    127ms
  5   -2.771714711969       -7.43       -3.78    2.0    135ms
  6   -2.771714715166       -8.50       -4.97    1.0   93.5ms
  7   -2.771714715246      -10.09       -5.25    2.0    108ms
  8   -2.771714715250      -11.47       -6.03    2.0    124ms
  9   -2.771714715250      -14.40       -6.90    1.0    106ms
 10   -2.771714715250   +  -13.88       -7.27    1.0    118ms
 11   -2.771714715250   +  -14.57       -7.99    1.0   98.8ms
 12   -2.771714715250      -13.97       -8.95    2.0    123ms


-0.00013457362592287763

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770493042054                   -0.52    9.0         
  2   -2.771778308192       -2.89       -1.32    1.0   88.2ms
  3   -2.771801557465       -4.63       -2.44    1.0    114ms
  4   -2.771801987887       -6.37       -3.14    1.0   91.4ms
  5   -2.771802074236       -7.06       -4.19    2.0    130ms
  6   -2.771802074402       -9.78       -4.54    1.0    127ms
  7   -2.771802074473      -10.14       -5.43    1.0   95.6ms
  8   -2.771802074476      -11.56       -5.92    2.0    111ms
  9   -2.771802074476      -13.11       -6.65    1.0    113ms
 10   -2.771802074476      -14.03       -7.08    2.0    127ms
 11   -2.771802074476      -14.65       -7.69    1.0    119ms
 12   -2.771802074476   +  -14.12       -8.25    2.0    115ms


0.017612221226326205

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457362592287763
Displaced dipole:  0.017612221226326205
Polarizability :   1.7746794852249084


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920944183e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766553317572e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852758676142e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694608179742e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088790849943e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.328307019220e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.882115553968e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.882115553968e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 9.677209709650e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.122306713959e-10
[ Info: GMRES linsolve in iter 2; step 3: normres = 1.676148945192e-12
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 1.676148945192e-12
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 1.676138844122983e-12
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.